In [ ]:
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import librosa
import librosa.display
import soundfile as sf
from pydub import AudioSegment
from IPython.display import Audio, display

# Load MP3 file and convert to mono audio
filename = 'Test Case 01 Vocal + Track 20240715.mp3'
data, sample_rate = librosa.load(filename, sr=None, mono=True)

# Design an FIR filter with an odd number of taps
numtaps = 101  # Use an odd number of taps

def octave_band_filter_bank(signal, sample_rate, num_bands):
    filters = []
    band_data = []
    nyquist = 0.5 * sample_rate
    low = 0.1  # Start from a small positive value to avoid issues with low being zero
    high = nyquist / (2 ** (num_bands - 1))

    print(f"Initial values: low={low}, high={high}, nyquist={nyquist}")

    for i in range(num_bands):
        print(f"Band {i+1}: low={low}, high={high}")

        if high > nyquist:
            high = nyquist

        # Ensure valid cutoff frequencies
        if low >= high:
            print(f"Warning: low ({low}) >= high ({high}). Skipping this band.")
            low = high
            high = low * 2 if low * 2 < nyquist else nyquist
            continue

        # Handle potential edge cases
        if low == 0:
            taps = sig.firwin(numtaps, 0.01 / nyquist, pass_zero=True)  # Prevent invalid zero cutoff
        elif high == nyquist:
            taps = sig.firwin(numtaps, low / nyquist, pass_zero=False, scale=False)
        else:
            taps = sig.firwin(numtaps, [low / nyquist, high / nyquist], pass_zero=False)

        filtered_signal = sig.lfilter(taps, 1.0, signal)
        filters.append(taps)
        band_data.append(filtered_signal)

        # Update frequency band range
        low = high
        high = low * 2 if low * 2 < nyquist else nyquist

    return filters, band_data

# Decompose the signal into 8 octave bands
num_bands = 8
filters, bands = octave_band_filter_bank(data, sample_rate, num_bands)

# Save each band to a separate MP3 file
def save_as_mp3(filename, data, sample_rate):
    temp_wav = filename.replace('.mp3', '.wav')
    sf.write(temp_wav, data, sample_rate)
    sound = AudioSegment.from_wav(temp_wav)
    sound.export(filename, format="mp3")

for i in range(num_bands):
    band_filename = f'band_{i+1}.mp3'
    save_as_mp3(band_filename, bands[i], sample_rate)
    print(f'Saved {band_filename}')
    # Display audio player in the notebook
    display(Audio(filename=band_filename))

# Reconstruct the signal by summing the bands without noise addition
reconstructed_signal = np.sum(bands, axis=0)

# Save the reconstructed signal as MP3
reconstructed_filename = 'reconstructed_signal.mp3'
save_as_mp3(reconstructed_filename, reconstructed_signal, sample_rate)
print(f'Saved {reconstructed_filename}')

# Display audio player for reconstructed signal
display(Audio(filename=reconstructed_filename))

# Visualize the original signal and the decomposed bands
plt.figure(figsize=(15, 10))

plt.subplot(num_bands + 1, 1, 1)
librosa.display.waveshow(data, sr=sample_rate)
plt.title('Original Signal')

for i in range(num_bands):
    plt.subplot(num_bands + 1, 1, i + 2)
    librosa.display.waveshow(bands[i], sr=sample_rate)
    plt.title(f'Band {i + 1}')

plt.tight_layout()
plt.show()

# Visualize the reconstructed signal
plt.figure(figsize=(10, 4))
librosa.display.waveshow(reconstructed_signal, sr=sample_rate)
plt.title('Reconstructed Signal')
plt.show()

In [ ]:
import numpy as np
import scipy.ndimage
import librosa
import soundfile as sf
from IPython.display import Audio

def harmonic_percussive_separation(y, sr, n_fft=2048, hop_length=512, win_length=None, window='hann', center=True, pad_mode='reflect'):
    if win_length is None:
        win_length = n_fft

    # Compute the Short-Time Fourier Transform (STFT)
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, center=center, pad_mode=pad_mode)
    
    # Obtain the magnitude and phase of the STFT
    magnitude, phase = librosa.magphase(D)
    
    # Initialize matrices for harmonic and percussive components
    harmonic = np.zeros_like(D)
    percussive = np.zeros_like(D)
    
    # Kernel size for median filtering
    kernel_size = (5, 5)
    
    # Apply median filtering to extract harmonic and percussive components
    harmonic_mag = scipy.ndimage.median_filter(magnitude, size=kernel_size)
    percussive_mag = magnitude - harmonic_mag
    
    # Construct harmonic and percussive spectrograms
    harmonic = harmonic_mag * phase
    percussive = percussive_mag * phase

    # Compute the Inverse Short-Time Fourier Transform (ISTFT)
    y_harmonic = librosa.istft(harmonic, hop_length=hop_length, win_length=win_length, window=window, center=center, dtype=np.float32)
    y_percussive = librosa.istft(percussive, hop_length=hop_length, win_length=win_length, window=window, center=center, dtype=np.float32)

    return y_harmonic, y_percussive

# Process each file and save the percussive component
for i in range(1, 9):
    # Load the audio file
    file_path = f'band_{i}.wav'
    y, sr = librosa.load(file_path, sr=None)
    
    # Perform harmonic and percussive separation
    # _, y_percussive = harmonic_percussive_separation(y, sr)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    
    # Save the percussive component
    output_path = f'projected_signal_band_{i}.wav'
    sf.write(output_path, y_percussive, sr)

    display(Audio(output_path))


In [ ]:
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
import numpy as np

def plot_waveform(wav_path, title):
    # Read the signal
    sample_rate, signal = wav.read(wav_path)

    # Create a time axis
    time = np.linspace(0, len(signal) / sample_rate, num=len(signal))

    # Plot the signal
    plt.figure(figsize=(15, 4))
    plt.plot(time, signal, label=title, color='blue')
    plt.title(f'{title} Waveform')
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.show()

# # Paths to the WAV files
# original_wav_path = 'original_signal_band_1.wav'
# projected_wav_path = 'projected_signal_band_1.wav'

# # Call the function to plot the waveforms
# plot_waveform(band_to_read, 'band')
# plot_waveform(original_wav_path, 'Original Signal')
# plot_waveform(projected_wav_path, 'Projected Signal')

In [ ]:
import numpy as np
import scipy.io.wavfile as wav
from IPython.display import Audio

# Bass Drum
# Read the two projected signal files
sample_rate_1, projected_signal_1 = wav.read('projected_signal_band_1.wav')
sample_rate_2, projected_signal_2 = wav.read('projected_signal_band_2.wav')

# Ensure the sample rates are the same
assert sample_rate_1 == sample_rate_2, "The sample rates of the two audio files must be the same"

# Ensure the lengths of the signals are the same
assert len(projected_signal_1) == len(projected_signal_2), "The lengths of the two audio files must be the same"

# Compute the sum of the signals
detect_bass_drum = projected_signal_1 + projected_signal_2

# Normalize the combined signal to avoid distortion (range should be -1 to 1)
detect_bass_drum = detect_bass_drum / np.max(np.abs(detect_bass_drum))

# Save the result as a new WAV file
wav.write('detect_bass_drum.wav', sample_rate_1, detect_bass_drum.astype(np.float32))

# Display the combined audio
print("Combined Bass Drum Signal:")
display(Audio('detect_bass_drum.wav'))

# Snare Drum
# Read the two projected signal files
sample_rate_3, projected_signal_3 = wav.read('projected_signal_band_3.wav')
sample_rate_4, projected_signal_4 = wav.read('projected_signal_band_4.wav')

# Ensure the sample rates are the same
assert sample_rate_3 == sample_rate_4, "The sample rates of the two audio files must be the same"

# Ensure the lengths of the signals are the same
assert len(projected_signal_3) == len(projected_signal_4), "The lengths of the two audio files must be the same"

# Compute the sum of the signals
detect_snare_drum = projected_signal_3 + projected_signal_4

# Normalize the combined signal to avoid distortion (range should be -1 to 1)
detect_snare_drum = detect_snare_drum / np.max(np.abs(detect_snare_drum))

# Save the result as a new WAV file
wav.write('detect_snare_drum.wav', sample_rate_3, detect_snare_drum.astype(np.float32))

# Display the combined audio
print("Combined Snare Drum Signal:")
display(Audio('detect_snare_drum.wav'))

# Cymbal
# Read the two projected signal files
sample_rate_7, projected_signal_7 = wav.read('projected_signal_band_7.wav')
sample_rate_8, projected_signal_8 = wav.read('projected_signal_band_8.wav')

# Ensure the sample rates are the same
assert sample_rate_7 == sample_rate_8, "The sample rates of the two audio files must be the same"

# Ensure the lengths of the signals are the same
assert len(projected_signal_7) == len(projected_signal_8), "The lengths of the two audio files must be the same"

# Compute the sum of the signals
detect_cymbal = projected_signal_7 + projected_signal_8

# Normalize the combined signal to avoid distortion (range should be -1 to 1)
detect_cymbal = detect_cymbal / np.max(np.abs(detect_cymbal))

# Save the result as a new WAV file
wav.write('detect_cymbal.wav', sample_rate_7, detect_cymbal.astype(np.float32))

# Display the combined audio
print("Combined Cymbal Signal:")
display(Audio('detect_cymbal.wav'))

In [ ]:
import numpy as np
import scipy.io.wavfile as wav
from scipy.signal import resample

def downsample_and_half_wave_rectify(input_file, output_file, downsample_factor):
    # Read the original signal
    sample_rate, signal = wav.read(input_file)

    # Normalize the signal if needed (ensure the signal values are in the range [-1, 1])
    if signal.dtype != np.float32:
        signal = signal / np.max(np.abs(signal))

    # Downsample the signal
    downsampled_signal = resample(signal, len(signal) // downsample_factor)

    # Apply half-wave rectification
    half_wave_rectified_signal = np.maximum(0, downsampled_signal)

    # Save the processed signal as a new WAV file
    wav.write(output_file, sample_rate // downsample_factor, half_wave_rectified_signal.astype(np.float32))

    return half_wave_rectified_signal

# Define the downsample factor
downsample_factor = 1  # For example, to halve the sample rate

# Process the bass drum signal
bass_drum_processed = downsample_and_half_wave_rectify('detect_bass_drum.wav', 'downsampled_half_wave_rectified_bass_drum.wav', downsample_factor)

# Process the snare drum signal
snare_drum_processed = downsample_and_half_wave_rectify('detect_snare_drum.wav', 'downsampled_half_wave_rectified_snare_drum.wav', downsample_factor)

# Process the cymbal signal
cymbal_processed = downsample_and_half_wave_rectify('detect_cymbal.wav', 'downsampled_half_wave_rectified_cymbal.wav', downsample_factor)

# Display the processed audio files
print("Processed Bass Drum Signal:")
display(Audio('downsampled_half_wave_rectified_bass_drum.wav'))

print("Processed Snare Drum Signal:")
display(Audio('downsampled_half_wave_rectified_snare_drum.wav'))

print("Processed Cymbal Signal:")
display(Audio('downsampled_half_wave_rectified_cymbal.wav'))

plot_waveform('downsampled_half_wave_rectified_bass_drum.wav', 'Bass Drum')
plot_waveform('downsampled_half_wave_rectified_snare_drum.wav', 'Snare Drum')
plot_waveform('downsampled_half_wave_rectified_cymbal.wav', 'Cymbal')

In [ ]:
import numpy as np
import scipy.io.wavfile as wav

def create_mask(signal, std_dev):
    # Create a mask based on the standard deviation
    mask = np.where(signal > 2 * std_dev, 1, 0)
    return mask

def process_and_create_mask(input_file, downsample_factor):
    # Read the processed signal
    sample_rate, signal = wav.read(input_file)
    
    # Compute the standard deviation of the signal
    std_dev = np.std(signal)
    
    # Create the mask
    mask = create_mask(signal, std_dev)
    
    return mask, sample_rate

# Define the downsample factor
downsample_factor = 8  # Assuming you want to use the same factor as before

# Process and create masks for each signal
bass_drum_mask, sample_rate_bass = process_and_create_mask('downsampled_half_wave_rectified_bass_drum.wav', downsample_factor)
snare_drum_mask, sample_rate_snare = process_and_create_mask('downsampled_half_wave_rectified_snare_drum.wav', downsample_factor)
cymbal_mask, sample_rate_cymbal = process_and_create_mask('downsampled_half_wave_rectified_cymbal.wav', downsample_factor)

# Save the masks as new audio files (optional, if you need to save them)
wav.write('bass_drum_mask.wav', sample_rate_bass, bass_drum_mask.astype(np.int16))
wav.write('snare_drum_mask.wav', sample_rate_snare, snare_drum_mask.astype(np.int16))
wav.write('cymbal_mask.wav', sample_rate_cymbal, cymbal_mask.astype(np.int16))

# Display the masks (optional, for verification)
print("Bass Drum Mask:")
plot_waveform('bass_drum_mask.wav', 'Bass Drum')

print("Snare Drum Mask:")
plot_waveform('snare_drum_mask.wav', 'Snare Drum')

print("Cymbal Mask:")
plot_waveform('cymbal_mask.wav', 'Cymbal')


In [ ]:
# Define r values
r_bass_drum = [1, 1, 1, 1, 1, 1, 0, 0]
r_snare_drum = [0, 1, 1, 1, 1, 1, 1, 1]
r_cymbal = [0, 0, 0, 0, 1, 1, 1, 1]

# Compute modulation coefficients w_k
w_signals = []

for k in range(8):
    r_values = [r_bass_drum[k], r_snare_drum[k], r_cymbal[k]]
    masks = [bass_drum_mask, snare_drum_mask, cymbal_mask]
    
    # Compute w_k = max(r_ki * alpha_i) for i = 1 to 3 at each time point
    w_k = np.maximum.reduce([r_value * mask for r_value, mask in zip(r_values, masks)])
    w_signals.append(w_k)

    # Save w_k as an audio file
    wav.write(f'modulation_coefficient_w_{k+1}.wav', sample_rate_bass, w_k.astype(np.float32))

# Display the results
for k in range(8):
    print(f"Modulation Coefficient w_{k+1}:")
    # display(Audio(f'modulation_coefficient_w_{k+1}.wav', rate=sample_rate_bass))
    plot_waveform(f'modulation_coefficient_w_{k+1}.wav', f'{k+1}')

In [ ]:
import numpy as np
import scipy.io.wavfile as wavfile
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt

def smooth_signal(signal, cutoff, fs, order=2):
    """
    Apply a low-pass Butterworth filter to smooth the signal.
    
    Parameters:
    - signal: Input signal (numpy array).
    - cutoff: Cutoff frequency for the low-pass filter.
    - fs: Sampling frequency of the signal.
    - order: Order of the Butterworth filter.
    
    Returns:
    - y: Smoothed signal.
    """
    normal_cutoff = cutoff / (0.5 * fs)
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, signal)
    return y

def normalize_signal(signal):
    """
    Normalize the signal to the range [0, 1].
    
    Parameters:
    - signal: Input signal (numpy array).
    
    Returns:
    - normalized_signal: Normalized signal.
    """
    min_val = np.min(signal)
    max_val = np.max(signal)
    normalized_signal = (signal - min_val) / (max_val - min_val)
    return normalized_signal

# Parameters
cutoff_freq = 20.0  # Cutoff frequency for smoothing, adjust as needed
filter_order = 2     # Filter order for smoothing

# Process each file
for k in range(1, 9):
    input_filename = f'modulation_coefficient_w_{k}.wav'
    output_filename = f'smoothed_modulation_coefficient_w_{k}.wav'
    
    # Load the WAV file
    sampling_rate, data = wavfile.read(input_filename)
    
    # Apply smoothing to the signal
    smoothed_data = smooth_signal(data, cutoff_freq, sampling_rate, order=filter_order)
    
    # Clip and convert the smoothed data to the appropriate format for saving
    smoothed_data = np.clip(smoothed_data, 0, 1)
    smoothed_data = (smoothed_data * 32767).astype(np.int32)

    # Normalize the smoothed data
    smoothed_data = normalize_signal(smoothed_data)
    
    # Save the smoothed signal as a new WAV file
    wavfile.write(output_filename, sampling_rate, smoothed_data)
    
    # Plot the original and smoothed signals
    plt.figure(figsize=(12, 6))
    
    # Plot the original signal
    plt.subplot(2, 1, 1)
    plt.plot(data, label='Original Signal', color='blue')
    plt.title(f'Original Signal - {input_filename}')
    plt.xlabel('Sample Index')
    plt.ylabel('Amplitude')
    plt.legend()
    
    # Plot the smoothed signal
    plt.subplot(2, 1, 2)
    plt.plot(smoothed_data, label='Smoothed Signal', color='red')
    plt.title(f'Smoothed Signal - {output_filename}')
    plt.xlabel('Sample Index')
    plt.ylabel('Amplitude')
    plt.legend()
    
    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal as signal

# Function to read the modulation coefficient signals
def read_modulation_coefficients(num_coefficients):
    modulation_coefficients = []
    for k in range(1, num_coefficients + 1):
        sample_rate, w_k = wav.read(f'smoothed_modulation_coefficient_w_{k}.wav')
        modulation_coefficients.append((sample_rate, w_k))
    return modulation_coefficients

# Function to read the band signals
def read_band_signals(num_bands):
    band_signals = []
    for i in range(1, num_bands + 1):
        sample_rate, band_signal = wav.read(f'projected_signal_band_{i}.wav')
        band_signals.append((sample_rate, band_signal))
    return band_signals

# Read the modulation coefficient signals
num_coefficients = 8
modulation_coefficients = read_modulation_coefficients(num_coefficients)

# Read the band signals
num_bands = 8
band_signals = read_band_signals(num_bands)

# Ensure all signals have the same length
min_length = min(len(band_signal[1]) for band_signal in band_signals)

# Compute the drum_extraction signal
drum_extraction = np.zeros(min_length, dtype=np.float32)

for k in range(num_coefficients):
    mc_sample_rate, mc = modulation_coefficients[k]
    _, band_signal = band_signals[k]
    
    # Upsample the modulation coefficient signal
    upsampled_mc = signal.resample(mc, len(band_signal))
    
    # Ensure the upsampled modulation coefficient has the same length as the band signal
    upsampled_mc = upsampled_mc[:min_length]
    band_signal = band_signal[:min_length]
    
    # Compute the product and accumulate
    drum_extraction += upsampled_mc * band_signal

# Normalize the drum_extraction signal if necessary
drum_extraction /= np.max(np.abs(drum_extraction))

# Save the drum_extraction signal as a new audio file
sample_rate = band_signals[0][0]  # Assuming all band signals have the same sample rate
wav.write('drum_extraction.wav', sample_rate, drum_extraction.astype(np.float32))

# Display the extracted drum signal
print("Extracted Drum Signal:") 
display(Audio('drum_extraction.wav', rate=sample_rate))
